In [3]:
import simpy
import random
import statistics

In [4]:
NUM_RECEPCIONISTAS = 1
NUM_MEDICOS = 2
TIEMPO_REGISTRO_PROMEDIO = 2.0
TIEMPO_CONSULTA_PROMEDIO = 7.0
TASA_LLEGADA_PACIENTES = 5.0
TIEMPO_SIMULACION = 120
tiempos_de_espera_totales = []


In [5]:
# =====================
# PROCESO PACIENTE
# =====================
def paciente(env, nombre, recepcionistas, medicos):
    """Simula el recorrido de un paciente por la clínica."""
    tiempo_llegada = env.now
    print(f"{nombre} llega a la clínica en el minuto {tiempo_llegada:.2f}")

    # --- Etapa 1: Registro ---
    with recepcionistas.request() as req:
        yield req
        print(f"{nombre} inicia registro en minuto {env.now:.2f}")
        duracion_registro = random.expovariate(1.0 / TIEMPO_REGISTRO_PROMEDIO)
        yield env.timeout(duracion_registro)

    # --- Etapa 2: Consulta ---
    with medicos.request() as req:
        yield req
        print(f"{nombre} inicia consulta en minuto {env.now:.2f}")
        duracion_consulta = random.expovariate(1.0 / TIEMPO_CONSULTA_PROMEDIO)
        yield env.timeout(duracion_consulta)

    # --- Salida ---
    tiempo_total = env.now - tiempo_llegada
    tiempos_de_espera_totales.append(tiempo_total)
    print(f"{nombre} sale en el minuto {env.now:.2f} (tiempo total: {tiempo_total:.2f} min)")


In [6]:
# =====================
# GENERADOR DE PACIENTES
# =====================
def generador_pacientes(env, recepcionistas, medicos):
    """Crea pacientes a intervalos aleatorios."""
    i = 0
    while True:
        yield env.timeout(random.expovariate(1.0 / TASA_LLEGADA_PACIENTES))
        i += 1
        env.process(paciente(env, f"Paciente {i}", recepcionistas, medicos))

In [7]:

# =====================
# BLOQUE PRINCIPAL
# =====================
env = simpy.Environment()
recepcionistas = simpy.Resource(env, NUM_RECEPCIONISTAS)
medicos = simpy.Resource(env, NUM_MEDICOS)

env.process(generador_pacientes(env, recepcionistas, medicos))
env.run(until=TIEMPO_SIMULACION)


Paciente 1 llega a la clínica en el minuto 4.08
Paciente 1 inicia registro en minuto 4.08
Paciente 1 inicia consulta en minuto 4.53
Paciente 2 llega a la clínica en el minuto 7.02
Paciente 2 inicia registro en minuto 7.02
Paciente 2 inicia consulta en minuto 7.18
Paciente 2 sale en el minuto 9.11 (tiempo total: 2.09 min)
Paciente 3 llega a la clínica en el minuto 17.00
Paciente 3 inicia registro en minuto 17.00
Paciente 3 inicia consulta en minuto 17.16
Paciente 4 llega a la clínica en el minuto 18.09
Paciente 4 inicia registro en minuto 18.09
Paciente 1 sale en el minuto 18.81 (tiempo total: 14.74 min)
Paciente 4 inicia consulta en minuto 18.81
Paciente 4 sale en el minuto 19.58 (tiempo total: 1.49 min)
Paciente 5 llega a la clínica en el minuto 20.83
Paciente 5 inicia registro en minuto 20.83
Paciente 6 llega a la clínica en el minuto 23.50
Paciente 5 inicia consulta en minuto 23.79
Paciente 6 inicia registro en minuto 23.79
Paciente 3 sale en el minuto 24.57 (tiempo total: 7.57 min)

In [8]:
# =====================
# ANÁLISIS FINAL
# =====================
num_pacientes = len(tiempos_de_espera_totales)
promedio = statistics.mean(tiempos_de_espera_totales)
maximo = max(tiempos_de_espera_totales)

print("\n=== RESULTADOS ===")
print(f"Pacientes atendidos: {num_pacientes}")
print(f"Tiempo promedio total: {promedio:.2f} minutos")
print(f"Tiempo máximo en clínica: {maximo:.2f} minutos")


=== RESULTADOS ===
Pacientes atendidos: 22
Tiempo promedio total: 10.85 minutos
Tiempo máximo en clínica: 26.95 minutos


La mayor parte del tiempo total se “va” en la consulta (por servicio largo/variable) y en esperas puntuales por médico.
Traducción brutalmente honesta: hoy el “peso” del sistema está del lado de médicos.